# Compare the use of Myopic optimization from Pypsa-Earth-Sec

This notebook aims at exploring the use of Myopic optimization from Pypsa-Earth-Sec.
The notebook reads:
- results of myopic optimization for a given list of planning horizons
- the corresponding overnight optimization results for the same scenarios

Appropriate plots are provided to show at each planning horizon:
- installed capacity, thus the capacity of power plants already built
- optimal capacity, thus the total optimized capacity of power plants
- capacity expansion, which is the difference between optimal and installed capacity
- energy balance
- the retired capacity at each time step

Sources: 
- network_analysis.ipynb: https://github.com/pypsa-meets-earth/documentation/blob/main/notebooks/viz/network_analysis.ipynb
- Plot capacity - map view: https://github.com/pypsa-meets-earth/documentation/blob/main/notebooks/viz/regional_transm_system_viz.ipynb
- Analyse energy potential: https://github.com/pypsa-meets-earth/documentation/blob/main/notebooks/build_renewable_profiles.ipynb
- Analyse energy generation: https://pypsa.readthedocs.io/en/latest/examples/statistics.html

The execution of the overnight optimizations are assumed to be performed using scenario management with the format, with the format `{country}_{planning_horizon}`.
The myopic and overnight optimizations are assumed to be performed into two different main repository, which optionally can be the same folder.

In [ ]:
root_myopic = "/data/davidef/gitsec_my/pypsa-earth-sec/"
root_overnight = "/data/davidef/gitsec_base/pypsa-earth-sec/"

## Import packages

In [ ]:
import yaml
import pypsa
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
import os
import xarray as xr
import cartopy

In [ ]:
# change current directory to parent folder
import os
import sys

if not os.path.isdir("pypsa-earth"):
    os.chdir("../..")
sys.path.append(os.getcwd()+"/pypsa-earth/scripts")

## Path settings
This section reads the config parameters from your config.yaml file and automatically reads the output of the optimization with those settings

In [ ]:
config = yaml.safe_load(open(root_myopic + "config.yaml"))

# Read config.yaml settings:
name_myopic = config["run"]
countries = config["countries"]
simpl = config["scenario"]["simpl"]
clusters = config["scenario"]["clusters"]
planning_horizons = config["scenario"]["planning_horizons"]
ll = config["scenario"]["ll"]
opts = config["scenario"]["opts"]
sopts = config["scenario"]["sopts"]
demand = config["scenario"]["demand"]
h2export = config["export"]["h2export"]
discountrate = config["costs"]["discountrate"]

# Ensure elements are strings and properly joined
simpl_str = str(simpl[0])
clusters_str = str(clusters[0])
ll_str = str(ll[0])
opts_str = str(opts[0])
sopts_str = str(sopts[0])
demand_str = str(demand[0])
h2export_str = str(h2export[0])
discountrate_str = str(discountrate[0])

# auxiliary function to get the scenario name using the myopic one as standard.
# For example: for planning_horizon=2030, NG_MYOPIC becomes NG_2030
def get_scenario_name(name_myopic, planning_horizon):
    """Scenario format {name}_{planning_horizon}"""
    str_split = name_myopic.split("_")
    return str_split[0] + "_" + str(planning_horizon)

# Auxiliary function to get the network path for the myopic configuration
def get_myopic_network(planning_horizon):
    return os.path.join(
        root_myopic,
        "results",
        name_myopic,
        "postnetworks",
        f"elec_s{simpl_str}_{clusters_str}_ec_l{ll_str}_{opts_str}_{sopts_str}_{planning_horizon}_{discountrate_str}_{demand_str}_{h2export_str}export.nc",
    )

# Auxiliary function to get the network path for the overnight configuration by planning horizon
def get_overnight_network(planning_horizon):
    name_scenario = get_scenario_name(name_myopic, planning_horizon)
    return os.path.join(
        root_overnight,
        "results",
        name_scenario,
        "postnetworks", # {clusters_str} , {ll_str} {sopts_str} {h2export_str}
        f"elec_s{simpl_str}_10_ec_lc1.0_{opts_str}_1H_{planning_horizon}_{discountrate_str}_{demand_str}_10export.nc",
    )

scenario_name = name_myopic
scenario_subpath = scenario_name + "/" if scenario_name else ""

# Country shape file
regions_onshore_path = root_myopic + f"pypsa-earth/resources/shapes/country_shapes.geojson"

## Energy system analysis setup - power and energy generation

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
regions_onshore = gpd.read_file(regions_onshore_path)
country_coordinates = regions_onshore.total_bounds[[0, 2, 1, 3]]
warnings.simplefilter(action='default', category=FutureWarning)

fp_myopic = get_myopic_network(planning_horizons[0])

n = pypsa.Network(fp_myopic)

## Data import check

Plot of the region of interest

In [ ]:
x_mean = (regions_onshore.total_bounds[0] + regions_onshore.total_bounds[2])/2
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection": ccrs.EqualEarth(x_mean)})
with plt.rc_context({"patch.linewidth": 0.}):
    regions_onshore.plot(
    ax=ax,
    facecolor="green",
    edgecolor="white",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0,
    )
ax.set_title(", ".join(regions_onshore.name.values))

## Analyse energy system

Analys the future generation capacity expansion of the energy system - bar chart

In [ ]:
# Auxiliary function to calculate optimal, installed, and capacity expansion capacities and energy balance
# for a given network n.
# It returns a pandas series for each whose name can contain an optional postfix.
def get_expansion_quantities(n, postfix=""):
    optimal_capacity = n.statistics.optimal_capacity(comps=["Generator"]).droplevel(0).div(1e3).rename("optimal" + postfix).drop("load", errors="ignore")
    installed_capacity = n.statistics.installed_capacity(comps=["Generator"]).droplevel(0).div(1e3).rename("installed" + postfix).drop("load", errors="ignore")
    generation_capacity_expansion = (optimal_capacity - installed_capacity).rename("expansion" + postfix).drop("load", errors="ignore")
    rename_cols = {
        '-': 'Load',
        'load': 'load shedding',
    }

    energy_balance = (
        n.statistics.energy_balance(comps=["Generator", "Store"])
        .loc[:, :, "AC"]
        .groupby("carrier")
        .sum()
        .div(1e6)
        .rename("balance" + postfix)
    )
    return optimal_capacity, installed_capacity, generation_capacity_expansion, energy_balance

Loop over the planning horizons and compare the myopic and overnight optimization results

In [ ]:
scenario_configs = (
    ["myopic - " + str(ph) for ph in planning_horizons] +
    ["overnight - " + str(ph) for ph in planning_horizons]
)

optimal_list = []
installed_list = []
expansion_list = []
balance_list = []

for scenario_name in scenario_configs:

    config_name, planning_horizon = scenario_name.split(" - ")

    if config_name.lower() == "myopic":
        n = pypsa.Network(get_myopic_network(planning_horizon))
    else:
        n = pypsa.Network(get_overnight_network(planning_horizon))
    optimal_capacity, installed_capacity, generation_capacity_expansion, energy_balance = get_expansion_quantities(n, f" - {scenario_name}")
    optimal_list.append(optimal_capacity)
    installed_list.append(installed_capacity)
    expansion_list.append(generation_capacity_expansion)
    balance_list.append(energy_balance)

# Merge the pandas series into dataframes
total_optimal = pd.concat(optimal_list, axis=1)
total_installed = pd.concat(installed_list, axis=1)
total_expansion = pd.concat(expansion_list, axis=1)
total_balance = pd.concat(balance_list, axis=1)

Define an auxiliary function to help the plotting of the dataframe calculated previously about capacity expansion

In [ ]:
def plot_expansion_dataframe(df, title_name):
    plot_df = df.copy().T

    colors = {key.lower(): value.lower() for key, value in config["plotting"]["tech_colors"].items()}
    colors["Combined-Cycle Gas"] = colors["ocgt"]
    colors["load"] = "pink"

    # color-matching
    color_list = []
    for col in plot_df.columns:
        original_name = col.lower()
        color = colors.get(original_name.lower(), 'red')
        color_list.append(color)

    fig, ax = plt.subplots()
    plot_df.plot.bar(stacked=True, ax=ax, title=title_name, color=color_list)
    handles, labels = ax.get_legend_handles_labels()
    nice_labels = plot_df.columns
    ax.legend(handles, nice_labels, bbox_to_anchor=(1, 0), loc="lower left", title=None, ncol=1)

    plt.show()

### Plot of Expansion of generation capacity

In [ ]:
plot_expansion_dataframe(total_expansion, "Expansion capacity [GW]")

### Plot of Installed capacity

In [ ]:
plot_expansion_dataframe(total_installed, "Installed capacity [GW]")

Plot of optimal capacity

In [ ]:
plot_expansion_dataframe(total_optimal, "Optimal capacity [GW]")

### Plot of energy balance

In [ ]:
plot_expansion_dataframe(total_balance, "Energy Balance in TWh")

In [ ]:
total_balance

#### Check installed capacity in following year corresponds to optimal in the previous step

In [ ]:
reorder_vector = [int(i/2) if i % 2 == 0 else len(total_optimal.columns) + int((i-1)/2) for i in range(2*len(total_optimal.columns))]
column_order = pd.concat([total_installed.columns.to_series(), total_optimal.columns.to_series()]).iloc[reorder_vector]

pd.concat([total_optimal, total_installed], axis=1).loc[:, column_order]

#### Get retiring assets by planning_horizon

In [ ]:
# read costs from pypsa-earth
costs = pd.read_csv(root_myopic + f"pypsa-earth/resources/costs.csv", index_col="technology")

lifetime = costs.query("parameter == 'lifetime'").value.rename("lifetime")
lifetime["gas"] = lifetime["OCGT"]  # gas tech is missing in costs.csv

total_retiring_list = []

# loop over the planning horizons to calculate the retiring capacity; skip the first planning horizon
for (i_pre, i_current) in zip(planning_horizons[:-1], planning_horizons[1:]):
    # read previous network
    n_pre = pypsa.Network(get_myopic_network(i_pre))

    subset_gen = n_pre.generators[n_pre.generators.carrier.isin(lifetime.index)]

    p_retiring = (
        subset_gen.groupby("carrier")
        .apply(
            lambda x: x.query(
                f"build_year < {i_current - lifetime[x.name.split('-')[0]]}"
            ).p_nom.sum()
        )
        .rename(f"retiring - {i_current}")
    ).div(1e3).mul(-1.)  # change sign to show negative values

    total_retiring_list.append(p_retiring)

total_retiring = pd.concat(total_retiring_list, axis=1)

In [ ]:
plot_expansion_dataframe(total_retiring, "Retired capacity [GW]")